# Recurrent Neural Network Regression task - Bike sharing

Bike sharing systems are new generation of traditional bike rentals where whole process from membership, rental and return has become automatic. Through these systems, a user is able to easily rent a bike from a particular position and return it at another place.

The dataset contains the hourly count of rental bikes between years 2011 and 2012 in the Capital Bikeshare system (Wasington DC) with the corresponding weather and seasonal information.

The goal of this task is to train a regressor to predict total counts of bike rentals based on the provided features for a given hour. 

## Data source
[http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset)

## Feature description
* **dteday** - date time stamot
* **season** - season (1: spring, 2: summer, 3: fall, 4: winter)
* **yr** - year (0: 2011, 1: 2012)
* **mnth** - month (1 to 12)
* **hr** - hour (0 to 23)
* **holiday** - 1 if the day is a holiday, else 0 (extracted from [holiday schedules](https://dchr.dc.gov/page/holiday-schedules))
* **weekday** - day of the week (0 to 6)
* **workingday** - is 1 if day is neither weekend nor holiday, else 0.
* **weathersit** 
    * 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
* **temp** - Normalized temperature in degrees of Celsius.
* **atemp** - Normalized feeling temperature in degrees Celsius.
* **hum** - Normalized relative humidity.
* **windspeed** - Normalized wind speed.
* **casual** - Count of casual users.
* **registered** - Count of registered users.
* **cnt** -  Count of total rental bikes including both casual and registered. This is the target value. 

In [1]:
import pandas as pd
data = pd.read_csv('../data/bikes.csv', sep=',')
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Recurrent Neural Network Regressor

Implement a recurrent neural network regressor. Sort the data by time stamp and deal with it as it was a time series. Be aware of using data from the past as test data.

In [2]:
data.sort_values(['dteday', 'hr'])
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


### Add some features from the past

Add the target feature from the previous hour.

In [3]:
cnt = data['cnt']
data['hist'] = cnt.shift(1)
data = data[1:]

X_all = data[['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit','temp', 'atemp', 'hum', 'windspeed', 'hist']]
y_all = data['cnt']

data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,hist
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40,16.0
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32,40.0
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13,32.0
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1,13.0
5,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1,1.0


### Sequences

Prepare train and test data sequences.

In [4]:
import numpy as np

window = 10

X_all = X_all.as_matrix()
y_all = y_all.as_matrix()

X_seq = []
y_seq = []
for i in range(window, len(X_all) + 1):
    X_seq.append(X_all[i-window: i])
    y_seq.append(y_all[i-1])

X_seq = np.asarray(X_seq)
y_seq = np.asarray(y_seq)

### Split the data into train and test data sets

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_seq, 
    y_seq,
    random_state=1,
    test_size=0.1)

#split_index = int(X_seq.shape[0]*0.9)
#X_train = X_seq[:split_index,:,:]
#X_test = X_seq[split_index:,:,:]
#y_train = y_seq[:split_index]
#y_test = y_seq[split_index:]

print('Train size: {}'.format(X_train.shape))
print('Test size: {}'.format(X_test.shape))

Train size: (15632, 10, 12)
Test size: (1737, 10, 12)


Standardize the features

In [6]:
from sklearn.preprocessing import StandardScaler

train_shape = X_train.shape
test_shape = X_test.shape

scaler = StandardScaler()
scaler.fit(X_train.reshape(train_shape[0]*train_shape[1], train_shape[2]))
X_train = scaler.transform(X_train.reshape(train_shape[0]*train_shape[1], train_shape[2])).reshape(train_shape)
X_test = scaler.transform(X_test.reshape(test_shape[0]*test_shape[1], test_shape[2])).reshape(test_shape)

### Training a regressor
Design and train a recurrent regression model with at least one [LSTM](https://keras.io/layers/recurrent/) layer. Use the [mean squared error](https://keras.io/losses/) loss function. Experiment with various architectures, [activation functions](https://keras.io/activations/).

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM

model = Sequential()

model.add(LSTM(32, input_shape=(window, 12)))
model.add(Dropout(0.15))
model.add(Dense(32))
model.add(Dense(1, activation='linear'))

Using TensorFlow backend.


Compile the model

In [8]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae'])

Train the model

In [9]:
model.fit(X_train, y_train,
          batch_size = 128, epochs = 100, verbose=1,
          validation_data=(X_test, y_test))

Train on 15632 samples, validate on 1737 samples
Epoch 1/100
15632/15632 [==============================] - 1s 75us/step - loss: 59671.0811 - mean_absolute_error: 172.3212 - val_loss: 49453.1345 - val_mean_absolute_error: 153.6284
Epoch 2/100
15632/15632 [==============================] - 1s 61us/step - loss: 36516.0421 - mean_absolute_error: 134.0032 - val_loss: 32268.5678 - val_mean_absolute_error: 125.2075
Epoch 3/100
15632/15632 [==============================] - 1s 61us/step - loss: 24920.1800 - mean_absolute_error: 110.7849 - val_loss: 22784.0009 - val_mean_absolute_error: 104.0766
Epoch 4/100
15632/15632 [==============================] - 1s 62us/step - loss: 18198.8585 - mean_absolute_error: 94.9132 - val_loss: 17440.1276 - val_mean_absolute_error: 93.6941
Epoch 5/100
15632/15632 [==============================] - 1s 62us/step - loss: 14397.8912 - mean_absolute_error: 84.9766 - val_loss: 13909.4437 - val_mean_absolute_error: 81.1901
Epoch 6/100
15632/15632 [====================

15632/15632 [==============================] - 1s 66us/step - loss: 1732.0040 - mean_absolute_error: 28.5208 - val_loss: 1464.4539 - val_mean_absolute_error: 26.0387
Epoch 47/100
15632/15632 [==============================] - 1s 64us/step - loss: 1744.3963 - mean_absolute_error: 28.6933 - val_loss: 1414.0400 - val_mean_absolute_error: 25.5285
Epoch 48/100
15632/15632 [==============================] - 1s 60us/step - loss: 1704.8966 - mean_absolute_error: 28.4127 - val_loss: 1409.3531 - val_mean_absolute_error: 25.3057
Epoch 49/100
15632/15632 [==============================] - 1s 60us/step - loss: 1659.2827 - mean_absolute_error: 28.1001 - val_loss: 1436.9547 - val_mean_absolute_error: 25.2996
Epoch 50/100
15632/15632 [==============================] - 1s 61us/step - loss: 1640.3980 - mean_absolute_error: 27.9588 - val_loss: 1396.5750 - val_mean_absolute_error: 25.2587
Epoch 51/100
15632/15632 [==============================] - 1s 68us/step - loss: 1673.9763 - mean_absolute_error: 28.0

Epoch 92/100
15632/15632 [==============================] - 1s 58us/step - loss: 1395.6780 - mean_absolute_error: 25.6362 - val_loss: 1244.5065 - val_mean_absolute_error: 23.6303
Epoch 93/100
15632/15632 [==============================] - 1s 59us/step - loss: 1339.1925 - mean_absolute_error: 25.2352 - val_loss: 1205.8985 - val_mean_absolute_error: 23.5869
Epoch 94/100
15632/15632 [==============================] - 1s 60us/step - loss: 1384.7510 - mean_absolute_error: 25.6884 - val_loss: 1225.3174 - val_mean_absolute_error: 23.4867
Epoch 95/100
15632/15632 [==============================] - 1s 60us/step - loss: 1329.4912 - mean_absolute_error: 25.0524 - val_loss: 1212.7804 - val_mean_absolute_error: 23.3923
Epoch 96/100
15632/15632 [==============================] - 1s 59us/step - loss: 1343.6306 - mean_absolute_error: 25.4282 - val_loss: 1257.8317 - val_mean_absolute_error: 23.8950
Epoch 97/100
15632/15632 [==============================] - 1s 60us/step - loss: 1367.7871 - mean_absolut

### Evaluate the models

Measure mean squared error and mean absolute error evaluation metrics on both train and test data sets. Compute the mean and standard deviation of the target values.

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

y_pred = model.predict(X_test)
print ("Test mean: {}, std: {}".format(np.mean(y_test), np.std(y_test)))
print("Test Root mean squared error: {:.2f}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("Test Mean absolute error: {:.2f}".format(mean_absolute_error(y_test, y_pred)))

Test mean: 196.9222797927461, std: 187.62398319999474
Test Root mean squared error: 33.92
Test Mean absolute error: 22.90


In [11]:
y_pred = model.predict(X_train)
print("Train Root mean squared error: %.2f"
      % np.sqrt(mean_squared_error(y_train, y_pred)))
print("Train Mean absolute error: %.2f"
      % mean_absolute_error(y_train, y_pred))

Train Root mean squared error: 30.13
Train Mean absolute error: 20.65
